In [3]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig

from kernels.abstract_base import TimeSeriesKernel, StaticKernel
from kernels.static_kernels import LinearKernel, RBFKernel, PolyKernel
from kernels.integral import StaticIntegralKernel
from kernels.sig_pde import SigPDEKernel
from kernels.sig_trunc import TruncSigKernel
from kernels.gak import GlobalAlignmentKernel, sigma_gak

from features.random_fourier import RBF_RandomFourierFeatures

In [4]:
#### now implement TRP_RFSF features

from kernels.sig_trunc import cumsum_shift1

@torch.jit.script
def calc_P_RFF(
        X: Tensor,
        rff_weights_m : Tensor,
        P_m: Tensor,
        D: int,
    ):
    """
    Intermediate step in the calculation of the TRP-RFSF features.
    See Algo 3 in https://arxiv.org/pdf/2311.12214.pdf.

    Args:
        X (Tensor): Tensor of shape (..., T, d) of time series.
        rff_weights_m (Tensor): Tensor of shape (d, D) of RFF weights.
        P_m (Tensor): Shape (2D, 2D) with i.i.d. standard Gaussians.
        D (int): RFF dimension.
    """
    #TODO This is wrong. need to do phi(x_t) - phi(x_{t-1})
    matmul = X @ rff_weights_m #shape (..., T, D)
    rff = torch.cat([torch.cos(matmul), 
                     torch.sin(matmul)], 
                     dim=-1) / D**0.5 #shape (..., T, 2D)
    U = rff.diff(dim=-2) @ P_m #shape (..., T-1, 2D)
    return U



@torch.jit.script
def tensorised_random_projection_features(
        X: Tensor,
        trunc_level: int,
        rff_weights: Tensor,
        P: Tensor,
    ):
    """
    Calculates the TRP-RFSF features for the given input tensor,
    when the underlying kernel is the RBF kernel. See Algo 3 in
    https://arxiv.org/pdf/2311.12214.pdf.

    Args:
        X (Tensor): Tensor of shape (..., T, d) of time series.
        trunc_level (int): Truncation level of the signature transform.
        rff_weights (Tensor): Tensor of shape (trunc_level, d, D) with
            independent RFF weights for each truncation level.
        P (Tensor): Shape (trunc_level, 2D, 2D) with i.i.d. standard 
            Gaussians.

    Returns:
        Tensor: Tensor of shape (trunc_level, ..., D) of TRP-RFSF features
            for each truncation level.
    """
    #first level
    D = rff_weights.shape[-1]
    V = calc_P_RFF(X, rff_weights[0], P[0], D) #shape (..., T-1, 2D)
    V = V / D**0.5
    levels = [V.sum(dim=-2)]

    #subsequent levels
    for m in range(1, trunc_level):
        U = calc_P_RFF(X, rff_weights[m], P[m], D) #shape (..., T-1, 2D)
        V = cumsum_shift1(V, dim=-2) * U #shape (..., T-1, 2D)
        levels.append(V.sum(dim=-2))
    
    return torch.stack(levels, dim=0)



class TRP_RFSF_Gaussian():
    def __init__(
            self,
            trunc_level: int, #signature truncation level
            n_features: int, #RBF RFF dimension/2
            sigma: float, #RBF parameter
            only_last : bool = False, #whether to use only the last level
            seed: Optional[int] = None,
        ):
        self.trunc_level = trunc_level
        self.n_features = n_features
        self.sigma = sigma
        self.only_last = only_last
        self.seed = seed
        self.has_initialized = False


    def _init_given_input(
            self, 
            X: Tensor
        ):
        """
        Initializes the random weights and biases for the TRP-RFSF 
        map for the RBF kernel. This is 'trunc_level' independent
        RFF weights, and (trunc_level, 2D, 2D) matrix of i.i.d. 
        standard Gaussians for the tensorized projection.

        Args:
            X (Tensor): Example tensor of shape (..., T, d) of timeseries.
        """
        # Get shape, dtype nad device info.
        d = X.shape[-1]
        device = X.device
        dtype = X.dtype
        gen = torch.Generator(device=device)
        # if self.seed is not None:
        #     gen.manual_seed(self.seed)
        # else:
        #     gen.seed()
        
        #initialize the RFF weights for each truncation level
        self.rff_weights = torch.randn(
                    self.trunc_level,
                    d,
                    self.n_features, 
                    generator=gen,
                    device=device,
                    dtype=dtype
                    ) / self.sigma
        
        #initialize the tensorized projection matrix for each truncation level
        self.P = torch.randn(self.trunc_level,
                             2*self.n_features, 
                             2*self.n_features,
                             device=device,
                             dtype=dtype,)

            
    def __call__(
            self,
            X:Tensor,
        ):
        """
        Computes the TRP-RFSF features for the given input tensor,
        mapping time series from (T,d) to (2*n_features)

        Args:
            X (Tensor): Tensor of shape (..., T, d).
        
        Returns:
            Tensor: Tensor of shape (trunc_level, ..., 2*n_features) or
                (..., 2*n_features) if self.only_last=True.
        """
        if not self.has_initialized:
            self._init_given_input(X)
            self.has_initialized = True
        
        features = tensorised_random_projection_features(
            X, self.trunc_level, self.rff_weights, self.P
            )
        #TODO stack with 1
        if self.only_last:
            return features[-1]
        else:
            return features

In [9]:
##############################################
#### TRP-RFSF features  vs  SigRBF kernel ####
##############################################

def trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 10
    trunc_level = 5
    n_features = 500
    sigma = 1.0
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(RBFKernel(sigma=sigma), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 1000
    res = []
    for i in range(MC_iter):
        trp = TRP_RFSF_Gaussian(trunc_level, n_features, sigma, only_last=True)
        feat_X = trp(X)
        feat_Y = trp(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

trp_vs_kernel()

exact
 tensor([[1.1892, 0.5117],
        [1.3060, 0.8059],
        [0.4007, 1.3128]], device='cuda:0')
mean
 tensor([[ 1.3668, -0.0729],
        [ 1.5496,  0.4705],
        [ 0.1584,  1.9141]], device='cuda:0')
example
 tensor([[ -4.1804,   8.7523],
        [  5.5932,  15.0179],
        [ -2.1607, -18.3409]], device='cuda:0')


In [6]:
############################################################
#### Test RBF_RandomFourierFeatures vs exact RBF kernel ####
############################################################
def rff_vs_exact_RBFKernel():
    N=3
    N2= 2
    d = 10
    sigma=1
    dtype = torch.float64
    # torch.manual_seed(1)
    X = torch.randn(N, d, dtype=dtype).to("cuda") /np.sqrt(d)
    Y = torch.randn(N2, d, dtype=dtype).to("cuda") / np.sqrt(d)

    # Exact RBF kernel
    k = RBFKernel(sigma=sigma)
    K = k(X, Y)

    # Approximate RBF kernel using RBF_RandomFourierFeatures
    N_MC = 10000
    res = []
    for i in range(N_MC):
        RFF = RBF_RandomFourierFeatures(n_features=1000,
                                        sigma=sigma,
                                        method="cos(x)sin(x)",
                                        # method = "cos(x + b)",
                                        )
        feat_X = RFF(X)
        feat_Y = RFF(Y)
        K_rff = feat_X @ feat_Y.T
        res.append(K_rff)
    K_rff = torch.mean(torch.stack(res), dim=0)

    print("K\n",K)
    print("K_rff\n",K_rff)
    print("diff\n", K-K_rff)
    print("diffmean\n", torch.mean(abs(K-K_rff)))
    # the RFF approach cant reproduce results smaller than 1e-5 for some reason
    
#rff_vs_exact_RBFKernel()

In [7]:
#####################################
######### FROM KSIG LIBRARY #########
#####################################
import numpy as np
import ksig

def ksig_readme():
    # Number of signature levels to use.
    n_levels = 5 
    normalize=False

    # Use 100 components in RFF and projection.
    n_components = 100

    # Instantiate RFF feature map.
    static_feat = ksig.static.features.RandomFourierFeatures(n_components=n_components)
    # Instantiate tensor random projections.
    proj = ksig.projections.TensorizedRandomProjection(n_components=n_components)

    # The RFSF-TRP feature map and kernel. Additionally to working as a callable for
    # computing a kernel, it implements a fit and a transform method.
    rfsf_trp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels, static_features=static_feat, projection=proj, normalize=normalize)

    # Generate 1000 sequences of length 200 with 100 features.
    n_seq, l_seq, n_feat = 3, 20, 10
    X = np.random.randn(n_seq, l_seq, n_feat) / np.sqrt(n_feat)

    # Fit the kernel to the data.
    rfsf_trp_kernel.fit(X)

    # Compute the kernel matrix as before.
    K_XX = rfsf_trp_kernel(X)  # K_XX has shape (1000, 1000).

    # GEnerate another array of 800 sequences of length 250 and 100 features.
    n_seq2, l_seq2 = 4, 20
    Y = np.random.randn(n_seq2, l_seq2, n_feat) / np.sqrt(n_feat)

    # Compute the kernel matrix between X and Y.
    # The kernel does not have to be fitted a second time.
    K_XY = rfsf_trp_kernel(X, Y)  # K_XY has shape (1000, 800)

    # Alternatively, we may compute features separately for X and Y. Under the hood,
    # this is what the call method does, i.e. compute features and take their inner product.
    P_X = rfsf_trp_kernel.transform(X)  # P_X has shape (1000, 501)
    P_Y = rfsf_trp_kernel.transform(Y)  # P_Y shape shape (800, 501)
    print("P_X", P_X)






    # Use the RBF kernel for vector-valued data as static (base) kernel.
    static_kernel = ksig.static.kernels.RBFKernel()
    order = 1
    sig_kernel = ksig.kernels.SignatureKernel(n_levels=n_levels, order=order, static_kernel=static_kernel,
                                              normalize=normalize)

    # Sequence kernels take as input an array of sequences of ndim == 3,
    # and work as a callable for computing the kernel matrix.
    K_XX_exact = sig_kernel(X)
    K_XY_exact = sig_kernel(X, Y)  
    print("\n\nTRP XX\n", K_XX)
    print("Exact XX\n", K_XX_exact)
    print("\n\nTRP XY\n", K_XY)
    print("Exact XY\n", K_XY_exact)

ksig_readme()

P_X [[ 1.          0.16938581 -0.01477683 ...  1.01439695 -0.35565706
  -1.06618399]
 [ 1.         -0.11358577 -0.05926521 ...  0.07448315  0.42757781
   0.77903039]
 [ 1.          0.11425649  0.04889685 ... -1.16861843  0.41891721
  -0.27036211]]


TRP XX
 [[166.69526262  -0.62267175 -11.47149645]
 [ -0.62267175 217.97679858   1.09777993]
 [-11.47149645   1.09777993 221.52034794]]
Exact XX
 [[237.99150333   6.14074744   4.75992553]
 [  6.14074744 424.11553794  13.11729886]
 [  4.75992553  13.11729886 452.31237726]]


TRP XY
 [[ 22.49616482  13.87929888  44.46568306   0.40705072]
 [  0.66147454   1.02535087  19.91034724  26.80736993]
 [-13.05435661 -10.52013625  -2.6786683  -19.48743619]]
Exact XY
 [[ 4.68778383  6.70630779  7.44599698  4.13166661]
 [ 9.89661104  8.67594951  9.53098565  7.29275137]
 [ 7.20272446  6.85685598  6.48670424 11.07097174]]


In [8]:
#########################
## KSIG test TRP vs DP ##
#########################
import ksig
import numpy as np

def trp_vs_dp():
    n_seq, l_seq, n_feat = 10, 150, 100
    X = np.random.randn(n_seq, l_seq, n_feat)
    n_levels = 10  # Number of signature levels to use.
    n_components = 100 # Use dimension of RFF map

    # Instantiate RFF feature map.
    static_feat = ksig.static.features.RandomFourierFeatures(n_components=n_components)
    trp_proj = ksig.projections.TensorizedRandomProjection(n_components=n_components)
    dp_proj = ksig.projections.DiagonalProjection()

    # The RFSF-TRP feature map and kernel. Additionally to working as a callable for
    # computing a kernel, it implements a fit and a transform method.
    trp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels, 
        static_features=static_feat, 
        projection=trp_proj
        )
    dp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels,
        static_features=static_feat,
        projection=dp_proj
        )
    trp_kernel.fit(X)
    dp_kernel.fit(X)


    # TIME IT
    import timeit
    def function_dp():
        dp_kernel(X, X)
    def function_trp():
        trp_kernel(X, X)
    execution_time_dp = timeit.timeit(function_dp, number=10)
    print("Execution time of dp\t:", execution_time_dp)
    execution_time_trp = timeit.timeit(function_trp, number=10)
    print("Execution time of trp\t:", execution_time_trp)
#trp_vs_dp()